## Домашнее задание по программированию

In [30]:
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [64]:
cancer = datasets.load_breast_cancer()

In [65]:
X = np.hstack((np.ones(len(cancer.target)).reshape(-1,1),cancer.data))

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, cancer.target, random_state=42)

### Аналитическое решение

In [125]:
class LinearRegression:
    def fit(self, X, y):
        self.w = np.linalg.inv(X.T @ X) @ X.T @ y
    def predict(self, X):
        return list(map(lambda x: 1 if x >=0 else 0, X @ self.w))

In [171]:
def accuracy(y, y_pred):
    print('The accuracy of the classifier is {0:.3f}'.format(accuracy_score(y, y_pred)))

In [172]:
alg = LinearRegression()

In [173]:
alg.fit(X_train, y_train)

In [174]:
y_pred = alg.predict(X_test)

In [175]:
accuracy(y_test, y_pred)

The accuracy of the classifier is 0.762


### SGD

Вам предстоит написать свой **Stohastic Gradient Descent**. Сначала ознакомьтесь с [лекциими для студентов МФТИ.](https://github.com/ml-mipt/ml-mipt-part1/blob/master/2018/lectures/Lecture_06_linear_models.pdf) В данной лекции приводится основная часть кода для SGD. Ваша задача состоит в том, чтобы дописать все недостающее, т.е. сделать так, чтобы метод *fit* находил оптимальные веса модели.